# Notebook traduit en français d'après celui du cours "Deep Learning" d'Alfredo Canziani de la New York University  : 
https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb

# Echo du signal

L'écho des signaux `n` étapes est un exemple de tâche synchronisée many-to-many (plusieurs à plusieurs).

In [2]:
from res.sequential_tasks import EchoData
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1);

In [3]:
batch_size = 5
echo_step = 3
series_length = 20_000
BPTT_T = 20

train_data = EchoData(
    echo_step=echo_step,
    batch_size=batch_size,
    series_length=series_length,
    truncated_length=BPTT_T,
)
train_size = len(train_data)

test_data = EchoData(
    echo_step=echo_step,
    batch_size=batch_size,
    series_length=series_length,
    truncated_length=BPTT_T,
)
test_size = len(test_data)

In [4]:
# Affichons les 20 premiers pas des premières séquences pour voir les données d'écho :
print('(Premiere sequence d entrée)  x:', *train_data.x_batch[0, :20], '... ')
print('(Premiere sequence cible) y:', *train_data.y_batch[0, :20], '... ')

(Premiere sequence d entrée)  x: 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 1 ... 
(Premiere sequence cible) y: 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 ... 


In [5]:
# des séquences différentes de batch_size sont créées :
print('x_batch:', *(str(d)[1:-1] + ' ...' for d in train_data.x_batch[:, :20]), sep='\n')
print('x_batch de taille:', train_data.x_batch.shape)
print()
print('y_batch:', *(str(d)[1:-1] + ' ...' for d in train_data.y_batch[:, :20]), sep='\n')
print('y_batch de taille:', train_data.y_batch.shape)

x_batch:
0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 1 ...
1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0 0 ...
0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 ...
1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 0 1 0 1 0 ...
0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 ...
x_batch de taille: (5, 20000)

y_batch:
0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 ...
0 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 ...
0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 ...
0 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 0 1 ...
0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 ...
y_batch de taille: (5, 20000)


In [6]:
# Pour utiliser les RNN, les données sont organisées en
# morceaux de taille [batch_size, T, feature_dim]
print('x_chunk:', *train_data.x_chunks[0].squeeze(), sep='\n')
print('Premier x_chunk de taille:', train_data.x_chunks[0].shape)
print()
print('y_chunk:', *train_data.y_chunks[0].squeeze(), sep='\n')
print('Premier y_chunk de taille:', train_data.y_chunks[0].shape)

x_chunk:
[0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 1]
[1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0 0]
[0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0]
[1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 0 1 0 1 0]
[0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1]
Premier x_chunk de taille: (5, 20, 1)

y_chunk:
[0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0]
[0 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1]
[0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1]
[0 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0]
Premier y_chunk de taille: (5, 20, 1)


In [7]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, rnn_hidden_size, output_size):
        super().__init__()
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = torch.nn.RNN(
            input_size=input_size,
            hidden_size=rnn_hidden_size,
            num_layers=1,
            nonlinearity='relu',
            batch_first=True
        )
        self.linear = torch.nn.Linear(
            in_features=rnn_hidden_size,
            out_features=1
        )

    def forward(self, x, hidden):
        x, hidden = self.rnn(x, hidden)  
        x = self.linear(x)
        return x, hidden

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
def train(hidden):
    model.train()
       
    correct = 0
    for batch_idx in range(train_size):
        data, target = train_data[batch_idx]
        data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device)
        optimizer.zero_grad()
        if hidden is not None: hidden.detach_()
        logits, hidden = model(data, hidden)
        loss = criterion(logits, target)
        loss.backward()
        optimizer.step()
        
        pred = (torch.sigmoid(logits) > 0.5)
        correct += (pred == target.byte()).int().sum().item()
        
    return correct, loss.item(), hidden

In [10]:
def test(hidden):
    model.eval()   
    correct = 0
    with torch.no_grad():
        for batch_idx in range(test_size):
            data, target = test_data[batch_idx]
            data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device)
            logits, hidden = model(data, hidden)
            
            pred = (torch.sigmoid(logits) > 0.5)
            correct += (pred == target.byte()).int().sum().item()

    return correct

In [11]:
feature_dim = 1 #puisque nous avons une série scalaire
h_units = 4

model = SimpleRNN(
    input_size=1,
    rnn_hidden_size=h_units,
    output_size=feature_dim
).to(device)
hidden = None
        
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [12]:
n_epochs = 5
epoch = 0

while epoch < n_epochs:
    correct, loss, hidden = train(hidden)
    epoch += 1
    train_accuracy = float(correct) / train_size
    print(f'Train Epoch: {epoch}/{n_epochs}, loss: {loss:.3f}, accuracy {train_accuracy:.1f}%')

#test    
correct = test(hidden)
test_accuracy = float(correct) / test_size
print(f'Test accuracy: {test_accuracy:.1f}%')

Train Epoch: 1/5, loss: 0.538, accuracy 59.5%
Train Epoch: 2/5, loss: 0.082, accuracy 87.1%
Train Epoch: 3/5, loss: 0.001, accuracy 100.0%
Train Epoch: 4/5, loss: 0.000, accuracy 100.0%
Train Epoch: 5/5, loss: 0.000, accuracy 100.0%
Test accuracy: 100.0%


In [13]:
# Essayons un peu d'écho
my_input = torch.empty(1, 100, 1).random_(2).to(device)
hidden = None
my_out, _ = model(my_input, hidden)
print(my_input.view(1, -1).byte(), (my_out > 0).view(1, -1), sep='\n')

tensor([[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
         0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
         0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
         1, 1, 0, 0]], dtype=torch.uint8)
tensor([[ True,  True, False, False,  True,  True,  True,  True, False, False,
          True,  True,  True,  True,  True,  True, False,  True,  True, False,
         False,  True, False, False,  True,  True,  True, False,  True, False,
         False,  True,  True, False,  True,  True,  True,  True, False, False,
          True,  True, False, False, False, False,  True,  True,  True, False,
          True, False, False,  True,  True, False,  True,  True, False,  True,
         False, False,  True,  True,  True, False,  True,  True, False,  True,
          True,  True,  True, False, False, False, False, False,  True,  True,
  